In [ ]:
import matplotlib.pyplot as plt
import bioframe 
import pandas as pd
import numpy as np 
import os
import json
from io import StringIO

import akita_utils
import pysam
import h5py
genome_open = pysam.Fastafile('/project/fudenber_735/genomes/mm10/mm10.fa')

SyntaxError: invalid syntax (1719275035.py, line 9)

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '-1'
import tensorflow as tf
print(tf.__version__)

import pysam
from basenji import dataset, seqnn, dna_io,stream
head_i = 1 #mouse
model_num = 1 #which fold to use


#base_dir = '/project/fudenber_735/backup/DNN_HiC/human-mouse_5-16-21/'
#model_dir = base_dir+"/f"+str(model_num)+"_c0/train/"

base_dir = '/project/fudenber_735/tensorflow_models/akita/v2/models/'
model_dir = base_dir+"/f"+str(model_num)+"c0/train/"
model_file  = model_dir+'/model'+str(head_i)+'_best.h5'


# model_dir = '/home1/fudenber/repositories/basenji/manuscripts/akita/'
# model_file = model_dir+'/model_best.h5'

params_file = model_dir+'/params.json'
with open(params_file) as params_open:
    params = json.load(params_open)
    params_model = params['model']
    params_train = params['train']
seq_length = params_model['seq_length']
params_model['verbose'] = False
seqnn_model = seqnn.SeqNN(params_model)
print('built')
seqnn_model.restore(model_file, head_i=head_i)
print('restored')

hic_diags = params_model['diagonal_offset']
try:
    target_crop = params_model['trunk'][-2]['cropping']
except:
    target_crop = params_model['target_crop']
target_length_cropped = int((seq_length//2048 - target_crop*2 - hic_diags)* ((seq_length//2048 - target_crop*2 - hic_diags) +1)/2) 
target_map_size = seq_length//2048  - target_crop*2 
triu_tup = np.triu_indices(target_map_size,2)
target_map_size, target_length_cropped, triu_tup[0].shape

2.6.2


2022-06-16 17:28:37.678832: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /spack/apps/linux-centos7-x86_64/gcc-8.3.0/python-3.9.2-uvcroioc4witkp6qf7mbebof5ix4wlb6/lib:/spack/apps/linux-centos7-x86_64/gcc-8.3.0/pmix-3.1.3-3sm6emyqaxapunh7rwbjvtaqoqe2e5z3/lib:/spack/apps/linux-centos7-x86_64/gcc-8.3.0/openmpi-4.0.2-ipm3dnvlbtxawpi4ifz7jma6jgr7mexq/lib:/spack/apps/linux-centos7-x86_64/gcc-8.3.0/openblas-0.3.8-2no6mfziiclwxb7lstxoos335gnhjpes/lib:/spack/apps/gcc/8.3.0/lib64
2022-06-16 17:28:37.678878: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-16 17:28:37.678919: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (d05-43.hpc.usc.edu): /proc/driver/nvidia/version does not exist
2022-06-16 17:28:37.679213:

built
restored


(512, 130305, (130305,))

In [6]:
background_file = base_dir + '../analysis/background_seqs.fa'
background_seqs = []
with open(background_file,'r') as f:
  for line in f.readlines():
    if '>' in line: continue
    background_seqs.append(dna_io.dna_1hot(line.strip())) 


In [9]:
seq_coords_df = akita_utils.prepare_insertion_tsv(
    h5_dirs = '/project/fudenber_735/tensorflow_models/akita/v2/analysis/permute_boundaries_motifs_ctcf_mm10_model*/scd.h5',
    score_key = 'SCD',
    flank_pad = 60, #how much flanking sequence around the sites to include
    weak_thresh_pct = 1, # don't use sites weaker than this, might be artifacts
    weak_num = 10 ,
    strong_thresh_pct = 99, # don't use sites weaker than this, might be artifacts
    strong_num = 10 ,
    save_tsv=None, # optional filename to save a tsv
)


orig 26544 filt 26515
29 duplicates removed
orig 26544 filt 26515
29 duplicates removed
annotating each site with boundary-wide scores


In [10]:
s = seq_coords_df.iloc[0]
seq_1hot_motif = dna_io.dna_1hot(genome_open.fetch(s.chrom, s.start, s.end).upper())
insert_length = len(seq_1hot_motif)
spacer_bp = 0
num_inserts = 6
multi_insert_length = num_inserts * (insert_length+spacer_bp)

offsets = []
for i in range(num_inserts):
    offsets.append( seq_length//2 - multi_insert_length//2 + i * (insert_length+spacer_bp))
offsets, seq_length//2

([654943, 655082, 655221, 655360, 655499, 655638], 655360)

In [11]:
all_inserts = []
for s in seq_coords_df.itertuples():
  for background_seq in background_seqs[0:1]:
    seq_1hot = background_seq.copy()
    seq_1hot_motif = dna_io.dna_1hot(genome_open.fetch(s.chrom, s.start, s.end).upper())
    if s.strand == '-': seq_1hot_motif = dna_io.hot1_rc(seq_1hot_motif)
    insert_length = len(seq_1hot_motif)
    for offset in offsets:
       seq_1hot[offset:offset+insert_length] = seq_1hot_motif
    all_inserts.append(seq_1hot)  
all_inserts = np.array( all_inserts)

In [ ]:
## predict for each insertion
pred = seqnn_model.predict(all_inserts, batch_size=10)

2022-06-16 17:35:33.340295: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [ ]:
plt.figure(figsize=(5*3,2*3))

target_ind = 0
vlim = .5
bin_mid = target_map_size//2
window = 50
for i in range(len(seq_coords_df)):
    insert_pred = pred[i]
    print(i, np.sqrt( (insert_pred**2).sum(axis=0)))

    plt.subplot(2,5, i+1)
    im = plt.matshow(
            from_upper_triu(  
            insert_pred[:,target_ind], target_map_size,hic_diags),
            vmin=-1*vlim, vmax=vlim, fignum=False,cmap='RdBu_r')
    plt.colorbar(im, fraction=0.046, pad=0.04)
    plt.title('genomic-scd: '+str(seq_coords_df['genomic_SCD'].values[i]) +'\n'+
              'insert-scd: '+str(  np.sqrt( (insert_pred**2).sum(axis=0)  ).mean() ) 
              ) 
    #plt.axis([ bin_mid  - window,bin_mid+window,bin_mid-window, bin_mid+window])
plt.tight_layout()
plt.show()